In [1]:
import re
import os
import string
import pickle
from transformers import AutoTokenizer

arabic_letters = ['ء', 'آ', 'أ', 'ؤ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ى', 'ي']
diacritics_list = ['ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ِ', 'ّ', 'ْ']
strings_potentially_undiacritized = ["ال", "ى", "ا","ي","و","آ","ٱ", "إ", "لل", "ٱل", "لِل", "لَل", "لا"]
strings_potentially_undiacritized = sorted(["ال", "ى", "ا","ي","و","آ","ٱ", "إ", "لل", "ٱل", "لِل", "لَل", "لا"], key = len, reverse = True)

mapping_harakat = {'ً': 'فتحتان',
 'ٌ': 'ضمتان',
 'ٍ': 'كسرتان',
 'َ': 'فتحة',
 'ُ': 'ضمة',
 'ِ': 'كسرة',
 'ّ': 'شدة',
 'ْ': 'سكون'}

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
numbers = '0123456789'
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations + numbers


/home/abderrahman.skiredj/.conda/envs/skiredj_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cleaning:
- if a word is partially diacritized while it shouldnt be: make -100 as a label to all its diacritics
- for each sentence:
    - if the nb tokens of preprocessed sentence < 512 ok
    - else: split it recursively until each split when preprocessed is of length < 512


In [2]:
import re

def remove_special_chars(word):
    return re.sub(f"[{re.escape(punctuations_list)}]", "", word)

def undiacritize(original_text):
    undiacritized_text = original_text
    for haraka in diacritics_list:
        undiacritized_text = undiacritized_text.replace(haraka, '')
    return undiacritized_text

In [3]:
def contain_diac(word):
    if any([x in word for x in diacritics_list]):
        return True
    return False

In [4]:
import re

# Arabic letters and diacritics
arabic_letters = ['ء', 'آ', 'أ', 'ؤ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ى', 'ي']
diacritics_list = ['ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ِ', 'ّ', 'ْ']

# Special cases and their automatic diacritization implications
special_cases_multiletters = {
    "ال": None,
    "لل": None,
    "في": None,
    "لا": None,
    "إي": None
}

special_cases_uniletters = {
    "إ": 'ِ',  # Kasra
    "آ": None,
    "ٱ": None,
    "ا": None,
    "ى": None
}

very_special_word = "عمرو"

def is_letter_diacritized(word, index):
    """Check if a letter at a given index is diacritized."""
    if index < len(word) - 1:
        # Check if next character is a diacritic
        return word[index + 1] in diacritics_list
    return False

def check_special_cases(word, index):
    """Check for special cases that do not require diacritization."""
    '''for case, diacritic in special_cases_multiletters.items():
        if word.startswith(case, index):
            return True, len(case)
    for case, diacritic in special_cases_uniletters.items():
        if word.startswith(case, index):
            return True, len(case)'''

    for case, diacritic in special_cases_multiletters.items():
        if undiacritize(word[index:]).startswith(case):
            len_case = len(case)
            if word[index+1] in diacritics_list:
                len_case += 1
            return True, len_case

    if index < len(word) - 2 and word[index + 1] in ["ا", "ى"] and word[index + 2]=="ل":
        # Letter followed by "ا" or "ى", considered diacritized
        return True, 3
    if index < len(word) - 3 and word[index + 1]=='َ' and word[index + 2] in ["ا", "ى"] and word[index + 3]=="ل":
        return True, 4
    if index < len(word) - 4 and word[index + 1]=='َ' and  word[index + 2]== 'ّ' and word[index + 3] in ["ا", "ى"] and word[index + 4]=="ل":
        return True, 5
    if index < len(word) - 4 and word[index + 1]== 'ّ' and  word[index + 2]=='َ' and word[index + 3] in ["ا", "ى"] and word[index + 4]=="ل":
        return True, 5

    if index < len(word) - 1 and word[index + 1] in ["ا", "ى"]:
        # Letter followed by "ا" or "ى", considered diacritized
        return True, 2
    if index < len(word) - 2 and word[index + 1]=='َ' and word[index + 2] in ["ا", "ى"]:
        return True, 3
    if index < len(word) - 2 and word[index + 1]=='ً' and word[index + 2] in ["ا", "ى"]:
        return True, 3
    if index < len(word) - 3 and word[index + 1]=='َ' and  word[index + 2]== 'ّ' and word[index + 3] in ["ا", "ى"]:
        return True, 4
    if index < len(word) - 3 and word[index + 1]== 'ّ' and  word[index + 2]=='َ' and word[index + 3] in ["ا", "ى"]:
        return True, 4

    if index < len(word) - 3 and word[index + 1]=='ً' and  word[index + 2]== 'ّ' and word[index + 3] in ["ا", "ى"]:
        return True, 4
    if index < len(word) - 3 and word[index + 1]== 'ّ' and  word[index + 2]=='ً' and word[index + 3] in ["ا", "ى"]:
        return True, 4

    if index > 1 and word[index -1] in ['َ' ,  'ّ'] and word[index] == "ا":
        return True, 1
        
    if index < len(word) - 2 and word[index + 1]=='ِ' and word[index + 2]=="ي":
        return True, 3
    if index < len(word) - 3 and word[index + 1]=='ِ' and word[index + 2]== 'ّ' and word[index + 3]=="ي":
        return True, 4
    if index < len(word) - 3 and word[index + 1]==  'ّ' and word[index + 2]=='ِ' and word[index + 3]=="ي":
        return True, 4

    if index > 1 and word[index -1] in [ 'ِ' ,  'ّ'] and word[index] == "ي":
        return True, 1
        
    if index < len(word) - 2 and word[index + 1]== 'ُ' and word[index + 2]=="و":
        return True, 3
    if index < len(word) - 3 and word[index + 1]== 'ُ' and word[index + 2]== 'ّ' and word[index + 3]=="و":
        return True, 4
    if index < len(word) - 3 and word[index + 1]== 'ّ' and word[index + 2]==  'ُ' and word[index + 3]=="و":
        return True, 4

    if index > 1 and word[index -1] in [ 'ُ', 'ّ' ]and word[index] == "و":
        return True, 1

    for case, diacritic in special_cases_uniletters.items():
        if undiacritize(word[index:]).startswith(case):
            return True, len(case)
    if undiacritize(word) == very_special_word and word[index] == "و":
        return True, 1
    return False, 0

def is_fully_diacritized(word):
    i = 0
    while i < len(word):
        if word[i] in arabic_letters:
            # Check if the current letter is within special cases
            special_case_found, jump = check_special_cases(word, i)
            if special_case_found:
                i += jump  # Skip the letters covered by the special case
                continue
            elif not is_letter_diacritized(word, i):
                return False  # Found an undiacritized letter outside special cases
        i += 1
    return True

# Examples to test the function
words = ["بِسْمِ", "الحَمْدُ", "ذَهبَ", "في", "مُسْتَشْفى", "للهِ", "إِنْ", "آمِنْتُمْ", "اللهُ", "الحَمْد", "بِسْم", "اللّهُ"] +  ["صِحَّتَهَا", "الرَّافِعِيُّ", "تَمْلِيكَ", "يُبِيحُ", "كَسَوَادٍ"] + ["بَكى", "بَكَى"] + ['أَبُو','حَتَّى',  'صَلَّى'] + ['وَالرَّهْنُ', "مَعْنًى", "لِلرَّدِّ", "وَ", "كَالطَّلْقِ", "كَالزَّمَانَةِ",  "وَالدَّوَاءِ", "لِلسُّكْرِ"] + ["مُذَكًّى", "تَعَالَى", "الْعِرَاقِيُّونَ", "تَعَالَى:", "الْحَضَرِيِّينَ", "فَوَلّى", "بِالْإِيلَاءِ", "صَلّى", "الْقُوَى"]
words = words + ["إيضَاحُهُ", "عُبَيدٍ", "عَمْرٍو"]
results = {word: is_fully_diacritized(word) for word in words}

for word, result in results.items():
    print(f"'{word}': {'Fully Diacritized' if result else 'Partially Diacritized'}")


'بِسْمِ': Fully Diacritized
'الحَمْدُ': Fully Diacritized
'ذَهبَ': Partially Diacritized
'في': Fully Diacritized
'مُسْتَشْفى': Fully Diacritized
'للهِ': Fully Diacritized
'إِنْ': Fully Diacritized
'آمِنْتُمْ': Fully Diacritized
'اللهُ': Partially Diacritized
'الحَمْد': Partially Diacritized
'بِسْم': Partially Diacritized
'اللّهُ': Fully Diacritized
'صِحَّتَهَا': Fully Diacritized
'الرَّافِعِيُّ': Fully Diacritized
'تَمْلِيكَ': Fully Diacritized
'يُبِيحُ': Fully Diacritized
'كَسَوَادٍ': Fully Diacritized
'بَكى': Fully Diacritized
'بَكَى': Fully Diacritized
'أَبُو': Fully Diacritized
'حَتَّى': Fully Diacritized
'صَلَّى': Fully Diacritized
'وَالرَّهْنُ': Fully Diacritized
'مَعْنًى': Fully Diacritized
'لِلرَّدِّ': Fully Diacritized
'وَ': Fully Diacritized
'كَالطَّلْقِ': Fully Diacritized
'كَالزَّمَانَةِ': Fully Diacritized
'وَالدَّوَاءِ': Fully Diacritized
'لِلسُّكْرِ': Fully Diacritized
'مُذَكًّى': Fully Diacritized
'تَعَالَى': Fully Diacritized
'الْعِرَاقِيُّونَ': Fully Diacritized
'تَعَ

In [6]:
index < len(word)-3

NameError: name 'index' is not defined

In [ ]:
word[index + 1]== 'ّ'

In [ ]:
word[index + 2] 

In [ ]:
index = 4
if index < len(word) - 3 and word[index + 1]=='َ' and  word[index + 2]== 'ّ' and word[index + 3] in ["ا", "ى"]:
    print(True, 4)
if index < len(word) - 3 and word[index + 1]== 'ّ' and  word[index + 2]=='َ' and word[index + 3] in ["ا", "ى"]:
    print(True, 4)

In [7]:
word[6]

'و'

In [8]:
check_special_cases(word, 6)

(True, 1)

1- take the test sentences, transform them to tokens, labels

2- keep the sentences with len input ids < 512

3- save the sentences with longer input ids

4- for PTCAD(0) split them and save them apart

In [5]:
file = open("tashkeela_50k_test/test.txt", "r")
test_sentences = file.readlines()

In [6]:
len(test_sentences)

2500

In [7]:
def diacritic_of(ar_letter_tuple, word):
    ar_letter, index = ar_letter_tuple
    diacritics = []
    i = index + 1
    while i < len(word):
        if word[i] in diacritics_list:
            diacritics.append(word[i])
        else:
            break
        i += 1
    return diacritics
    
def bert_transform_word(word):
    # Get the list of Arabic letters in the word
    transformed_word = {"tokens":[], "labels":[]}
    ar_letters = [c for c in word if c in arabic_letters]
    if not is_fully_diacritized(word):
        transformed_word['tokens'] = [undiacritize(word)]+len(ar_letters)*['[MASK]']
        transformed_word['labels'] = ['X'] + len(ar_letters)*['X']
        return transformed_word
    if len(ar_letters) <= 0:
        transformed_word['tokens'] = [word]
        transformed_word['labels'] = ['X']
        return transformed_word
    ar_letters = [(word[i], i) for i in range(len(word)) if word[i] in arabic_letters]
    # Get the list of diacritics associated with each Arabic letter in the word
    diacritics_list_word = [diacritic_of(ar_letter_tuple, word) for ar_letter_tuple in ar_letters]
    result_list = []
    for diacritics in diacritics_list_word:
        if diacritics == []:
            result_list.append(['تطويل'])
        else:
            result_list.append([mapping_harakat[diacritic] for diacritic in diacritics])
    transformed_word['tokens'].append(undiacritize(word))
    for haraka in result_list:
        transformed_word['tokens'].append('[MASK]')
    transformed_word['labels'].append('X')
    transformed_word['labels'].extend([' '.join(x) for x in result_list])
    return transformed_word
    
def bert_transform_sentence(sentence):
    transformed_sentence = {"tokens":[], "labels":[]}
    words = sentence.split(' ')
    for word in words:
        transformed_word = bert_transform_word(word)
        transformed_sentence['tokens'].extend(transformed_word['tokens'])
        transformed_sentence['labels'].extend(transformed_word['labels'])
    return transformed_sentence

def bert_seq_length_of(s, tokenizer):
    return len(tokenizer(''.join(bert_transform_sentence(s)['tokens']))['input_ids'])

In [8]:
bert_transform_word("وَ"), bert_transform_word("كَ")

({'tokens': ['و', '[MASK]'], 'labels': ['X', 'فتحة']},
 {'tokens': ['ك', '[MASK]'], 'labels': ['X', 'فتحة']})

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('AbderrahmanSkiredj1/arabertv02_tashkeel_fadel')

In [10]:
short_test_sentences = [x for x in test_sentences if bert_seq_length_of(x, tokenizer)<511]
len(short_test_sentences)

Token indices sequence length is longer than the specified maximum sequence length for this model (744 > 512). Running this sequence through the model will result in indexing errors


2202

In [11]:
long_sentences = [x for x in test_sentences if x not in short_test_sentences]
len(long_sentences)

298

In [17]:
short_test_sentences_transformed = [bert_transform_sentence(sentence) for sentence in short_test_sentences]
#short_test_sentences_transformed[0]

In [12]:
long_test_sentences_transformed = [bert_transform_sentence(sentence) for sentence in long_sentences]

In [13]:
from random import choice
choice(long_test_sentences_transformed)

{'tokens': ['13026',
  '-',
  'حدثنا',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'بهز',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'حدثنا',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'سليمان',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'بن',
  '[MASK]',
  '[MASK]',
  'المغيرة',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'عن',
  '[MASK]',
  '[MASK]',
  'ثابت',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'عن',
  '[MASK]',
  '[MASK]',
  'أنس',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'قال',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'مات',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'ابن',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'لأبي',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'طلحة',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'من',
  '[MASK]',
  '[MASK]',
  'أم',
  '[MASK]',
  '[MASK]',
  'سليم',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'فقالت',
  '[MASK]',
  '[MASK]

In [14]:
from datasets import Dataset

dataset_dict = {"tokens": [x["tokens"] for x in long_test_sentences_transformed], "labels": [x["labels"] for x in long_test_sentences_transformed]}
dat = Dataset.from_dict(dataset_dict)
dat

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 298
})

In [20]:
from datasets import Dataset

dataset_dict = {"tokens": [x["tokens"] for x in short_test_sentences_transformed], "labels": [x["labels"] for x in short_test_sentences_transformed]}
dat = Dataset.from_dict(dataset_dict)
dat

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 2202
})

In [15]:
len(dat[10]['tokens'])

587

In [16]:
dat.save_to_disk("test_datasets_processed/fadel_test_long_sentences")

Saving the dataset (1/1 shards): 100%|██████████| 298/298 [00:00<00:00, 28584.88 examples/s]


In [21]:
#dat.save_to_disk("test_datasets_processed/fadel_test_short_sentences")

Saving the dataset (1/1 shards): 100%|██████████| 2202/2202 [00:00<00:00, 183458.62 examples/s]


In [27]:
MAX_LENGTH = 511
def is_sentence_of_acceptable_length(s):
    return bert_seq_length_of(s, tokenizer) < MAX_LENGTH

def split_sentence(sentence):
    sub_sentences = []
    current_sentence = ""
    for word in sentence.split(" "):
        if is_sentence_of_acceptable_length(current_sentence + " " + word):
            current_sentence += " " + word
        else:
            sub_sent_to_append = current_sentence.strip()
            sub_sentences.append(sub_sent_to_append)
            current_sentence = word
    sub_sentences.append(current_sentence.strip())
    return sub_sentences

In [28]:
long_sentences_splitted = [split_sentence(x) for x in long_sentences]
long_sentences_splitted = [x for y in long_sentences_splitted for x in y]
len(long_sentences), len(long_sentences_splitted)

(298, 659)

In [30]:
a = choice(long_sentences_splitted)
print(a)

فِيمَنْ قَالَ لِامْرَأَتِهِ : أَنْتِ طَالِقٌ إنْ شِئْتِ أَوْ لِعَبْدِهِ أَنْتَ حُرٌّ إذَا قَدِمَ فُلَانٌ قُلْتُ : أَرَأَيْتَ لَوْ قَالَ رَجُلٌ لِامْرَأَتِهِ أَنْتِ طَالِقٌ إذَا شِئْت قَالَ : قَالَ مَالِكٌ : إنَّ الْمَشِيئَةَ لَهَا وَإِنْ قَامَتْ مِنْ مَجْلِسِهَا ذَلِكَ تُوقَفُ ، فَتَقْضِي أَوْ تَتْرُكُ فَإِنْ هِيَ تَرَكَتْهُ فَجَامَعَهَا قَبْلَ أَنْ تُوقَفَ أَوْ تَقْضِيَ فَلَا شَيْءَ لَهَا وَقَدْ بَطَلَ مَا كَانَ فِي يَدَيْهَا مِنْ ذَلِكَ قَالَ ابْنُ الْقَاسِمِ : وَإِنَّمَا قُلْتُ لَكَ فِي الرَّجُلِ الَّذِي يَقُولُ لِامْرَأَتِهِ أَنْتِ طَالِقٌ إنْ شِئْتِ أَنَّ ذَلِكَ بِيَدِهَا حَتَّى تُوقَفَ وَإِنْ تَفَرَّقَا مِنْ مَجْلِسِهِمَا ؛ لِأَنَّ مَالِكًا قَدْ تَرَكَ قَوْلَهُ الْأَوَّلَ فِي التَّمْلِيكِ وَرَجَعَ إلَى أَنْ قَالَ ذَلِكَ بِيَدِهَا حَتَّى تُوقَفَ ، فَهُوَ أَشْكَلُ مِنْ التَّمْلِيكِ ؛ لِأَنَّ


In [31]:
long_sentences_splitted.index(a)

234

In [33]:
long_sentences_splitted[236]

'أَقُولُ : وَمَعَ ذَلِكَ فَيُمْكِنُ تَوْجِيهُهُ بِأَنَّ قَوْلَهُ لِمَالِكِهَا بَدَلٌ مِنْ لِهَذِهِ الدَّابَّةِ ( قَوْلُهُ : فَإِنْ أَرَادَ غَيْرَهُ ) أَيْ كَأَنْ قَالَ : أَرَدْت مَنْ انْتَقَلَتْ مِنْهُ إلَى مَنْ هِيَ تَحْتَ يَدِهِ الْآنَ وَإِنْ طَالَتْ مُدَّةُ كَوْنِهَا فِي مِلْكِ مَنْ هِيَ تَحْتَ يَدِهِ ( قَوْلُهُ : وَلَوْ لَمْ يَقُلْ لِمَالِكِهَا ) بَلْ قَالَ عَلَيَّ بِسَبَبِ هَذِهِ الدَّابَّةِ ( قَوْلُهُ : لِمَالِكِهَا حَالًّا ) أَيْ بَلْ وَلَا لِمَالِكِهَا مُطْلَقًا لِجَوَازِ أَنْ تَكُونَ فِي يَدِهِ بِنَحْوِ إعَارَةٍ أَوْ غَصْبٍ فَأَتْلَفَتْ شَيْئًا فَهُوَ مَضْمُونٌ عَلَيْهِ لِمَالِكِهِ لَا لِمَالِكِهَا فَيُسْتَفْسَرُ وَيُعْمَلُ بِتَفْسِيرِهِ ا ه سم عَلَى حَجّ عَنْ شَرْحِ الْبَهْجَةِ بِالْمَعْنَى ( قَوْلُهُ : لِأَنَّهَا ) أَيْ الْبَلَدِ وَقَوْلُهُ ثُمَّ اُسْتُرِقَّ أَيْ الْحَرْبِيُّ ( قَوْلُهُ : فَإِنْ عَتَقَ فَلَهُ'

In [34]:
long_test_sentences_splitted_transformed = [bert_transform_sentence(sentence) for sentence in long_sentences_splitted]
from datasets import Dataset

dataset_dict = {"tokens": [x["tokens"] for x in long_test_sentences_splitted_transformed], "labels": [x["labels"] for x in long_test_sentences_splitted_transformed]}
dat = Dataset.from_dict(dataset_dict)
dat

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 659
})

In [35]:
dat[500]

{'tokens': ['قال',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  ':',
  'وإذا',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'اشترى',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'امرأته',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '،',
  'وقد',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'ولدت',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'فأعتقها',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'وتزوجها',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '،',
  'ثم',
  '[MASK]',
  '[MASK]',
  'جاءت',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'بولد',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'لستة',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'أشهر',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'منذ',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'تزوجها',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'فنفاه',
  '[MASK]'

In [36]:
dat.save_to_disk('test_datasets_processed/fadel_test_long_sentences_splitted')

Saving the dataset (1/1 shards): 100%|██████████| 659/659 [00:00<00:00, 64792.46 examples/s]


In [37]:
from datasets import load_from_disk, concatenate_datasets
dat2 = load_from_disk('test_datasets_processed/fadel_test_short_sentences')
daat = concatenate_datasets([dat2, dat])
daat

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 2861
})

In [38]:
daat.save_to_disk('test_datasets_processed/fadel_test_sentences')

Saving the dataset (1/1 shards): 100%|██████████| 2861/2861 [00:00<00:00, 124770.25 examples/s]


In [1]:
from datasets import load_from_disk
short = load_from_disk('test_datasets_processed/fadel_test_short_sentences')
long = load_from_disk('test_datasets_processed/fadel_test_long_sentences_splitted')
all = load_from_disk('test_datasets_processed/fadel_test_sentences')
short.push_to_hub('AbderrahmanSkiredj1/fadel_test_short_sentences_as_token_classif', private = True)
long.push_to_hub('AbderrahmanSkiredj1/fadel_test_long_sentences_splitted_as_token_classif', private = True)
all.push_to_hub('AbderrahmanSkiredj1/fadel_test_sentences_sizeok_as_token_classif', private = True)

/home/abderrahman.skiredj/.conda/envs/skiredj_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


## Draft

In [ ]:
word = "الْحَضَرِيِّينَ"
i = 0
while i < len(word):
    if word[i] in arabic_letters:
        print(word[i])
        # Check if the current letter is within special cases
        special_case_found, jump = check_special_cases(word, i)
        print(special_case_found, jump, i)
        if special_case_found:
            i += jump  # Skip the letters covered by the special case
            continue
        elif not is_letter_diacritized(word, i):
            print(False)  # Found an undiacritized letter outside special cases
    i += 1

In [15]:
["صِحَّتَهَا", "الرَّافِعِيُّ", "تَمْلِيكَ", "يُبِيحُ", "كَسَوَادٍ"]

False

In [67]:
tocheck = [x for x in short_test_sentences_transformed if any([contain_diac(y) for y in x["tokens"]])]

In [68]:
len(tocheck)

183

In [63]:
['وَالرَّهْنُ', "مَعْنًى", "لِلرَّدِّ", "وَ", "كَالطَّلْقِ", "كَالزَّمَانَةِ",  "وَالدَّوَاءِ", "لِلسُّكْرِ"]

['وَالرَّهْنُ', 'مَعْنًى']

In [45]:
["الْإِيضَاحِ", "تَعَالَى", "إيثَارُهُ", "اللَّيَالِي"]

['الْإِيضَاحِ']

In [ ]:
["مُذَكًّى", "تَعَالَى", "الْعِرَاقِيُّونَ", "تَعَالَى:", "الْحَضَرِيِّينَ", "فَوَلّى", "بِالْإِيلَاءِ", "صَلّى", "الْقُوَى"]

In [ ]:
["إيضَاحُهُ", "عُبَيدٍ", "عَمْرٍو"]

In [62]:
from random import choice
chosen = choice(tocheck)
print("\n".join([x for x in chosen['tokens'] if contain_diac(x)]))

عَمْرٍو
عَمْرٌو
عَمْرٍو
عَمْرٌو
وَلِعَمْرٍو


In [56]:
chosen

{'tokens': ['(',
  'وَ',
  ')',
  'إن',
  '[MASK]',
  '[MASK]',
  'رجع',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'بعضهم',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '(',
  'بعد',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'حد',
  '[MASK]',
  '[MASK]',
  ')',
  'مشهود',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'عليه',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '(',
  'يحد',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'راجع',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  ')',
  'عن',
  '[MASK]',
  '[MASK]',
  'شهادته',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '(',
  'فقط',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  ')',
  '؛',
  'أي',
  '[MASK]',
  '[MASK]',
  ':',
  'دون',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'من',
  '[MASK]',
  '[MASK]',
  'لم',
  '[MASK]',
  '[MASK]',
  'يرجع',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '؛',
  'لأن',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  'إقامة',
  '[MASK]',
  '[MASK]',
  '[MASK

In [55]:
is_fully_diacritized('وَ'), is_fully_diacritized("وَ")

(True, True)